In [1]:
from pysjef import Project, DirectoryNode, select, all_completed
from pysjef_molpro import no_errors
from multiprocessing.dummy import Pool
import numpy
import matplotlib.pyplot as plt
from operator import methodcaller

# Two state PES of C2H4 dihedral

Two state calculation with CASSCF(2,2) followed by MRCI

In [2]:
root = DirectoryNode('scan')

In [3]:
p_ref = root.add_child('d0', suffix='molpro')
p_ref.write_input("""file,2,mrci.wf
nosym
geometry={{
    H1
    C1 H1 rch
    C2 C1 rcc H1 A
    H2 C2 rch C1 A H1 D
    H3 C2 rch C1 A H2 180
    H4 C1 rch C2 A H1 180}}
rch = 2.05
rcc = 2.85
A = 121.5
D = 90.0
basis = cc-pVTZ
rhft; AVAS; CENTER, 2, 2p; CENTER, 3, 2p;
put,molden,rhft1.molden
{casscf; closed, 7; occ, 9}
put,molden,cas1.molden
optg; inactive, D
{casscf, so_sci; closed, 7; occ, 9; wf; state, 2}
put,molden,cas2.molden
{mrci; core, 4; wf; state, 2;}""")
p_ref.write_file('molpro.rc', content='-W .')
p_ref.run(wait=True)
if p_ref.errors(): raise RuntimeError('reference job failed')

In [4]:
grid = numpy.linspace(0, 90, 21)[::-1]
for i in range(1, grid.size):
    p = root.child(name=f'd{i-1}').copy(f'd{i}', force=True)
    p.write_input(f"""file,2,mrci.wf
        D = {grid[i]}
        {{casscf; closed, 7; occ, 9}}
        put,molden,cas1.molden
        optg; inactive, D
        {{casscf, so_sci; closed, 7; occ, 9; wf; state, 2}}
        put,molden,cas2.molden
        {{mrci; core, 4; wf; state, 2;}}""")
    p.run(wait=True)
    if p.errors(): raise RuntimeError(f'job failed index={i}, D={D}')
    root.add_child_node(p, replace=True)

In [5]:
%matplotlib notebook
energies = root.select('//property[name=Energy, method=MRCI].value')
e1, e2 = energies[::2], energies[1::2]
n = len(e1)
cc = root.select('//variable[name=rcc]/value..')
ch = root.select('//variable[name=rch]/value..')
ang = root.select('//variable[name=A]/value..')
#print('rCC = '+', '.join([f'{x:.2f}' for x in cc]))
plt.plot(grid[:n], e1, grid[:n], e2)
plt.legend(['Ground state','Excited state'])
plt.xlabel('HCCH dihedral angle (degrees)')
plt.ylabel('Energy (Eh)')
plt.show()

<IPython.core.display.Javascript object>

In [ ]:
with Pool(processes=8) as pool:
    pool.map(methodcaller('run', wait=True), root.children, 1)
print(f'jobs completed, {all_completed(root.children)}, with no errors, {no_errors(root.children)}')